## Functions and basic types

We will continue the RPG game from lesson 3 and dive deeper into functions and basic types.  

Python functions have several special abilities and are defined as follows:

- Can take default values for parameters
- Can specify keyword at the calling of a function
- Can take positional only parameters using `/`
- Can take keyword only parameters using `*`
- Can pass in a list for positional args with the `*identifier` (typically `*args`) operator
    - If specified, must come before the `*` marker
- Can pass in a dictionary for keyword args using the `**identifier` (typically `**kwargs`) operator
    - if `*args` is used, must come after it

> All these rules can make it pretty complicated pretty fast.  I recommend sticking to a subset of these.  It is often
> useful to specify by kw-only even if more verbose.  But to show you how complicated it can get here are some examples
> that we will progressively expand on

## A word on types

In python, types are purely optional:

- They are are validated by a static type checker (in your IDE or as part of a CI check) 
- At runtime, the python interpreter does not use any of this information
    - The annotations are available with reflection though
    - a few python libraries actually require type annotations like FastApi and Pydantic.

However, you should _always_ use types, even if they optional.

## Simple function

- starts with keyword `def`
- followed by the name of the function
- has a list of parameters enclosed in parenthesis
    - a parameter may be a `name: type` or `name: type = default_value`
    - the latter is called a _keyword argument_,  _keyword parameter_, or rarely, a `named argument`
- after the closing `)` it is followed by ` -> return_type:`
- followed by an optional triple quoted string called a docstring
- lastly the body of the function

> Parameters vs arguments
>
> In python, the two are typically used synonymously.  I will take the mojo convention that parameters are compile time
> or _definition_ time names and values, and arguments are the keyword names or values passed in at runtime.  This will
> help you understand Parameterization in mojo when I get to that

In [ ]:
import random as rand

def die(size: int) -> int:
    """Generates a random number from 1 - size (inclusize)

    Parameters
    ----------
    size : int
        The size of the die (eg, 4, 6, 8, 10, 12, 20, etc)

    Returns
    -------
    int
        the random result of the roll
    """
    return rand.randint(1, size)

In [ ]:
# Can call it without naming the variable
print(die(20))
# or by naming the variable
die(size=10)

## Compared to Java

So a couple of differences should stand out:

- functions are standalone, they don't have to be members of a class
- Java has no equivalent of default parameters (uses overloaded methods instead)
    - starting with 3.10, python has a form of Java's overloaded methods
- python does not have checked exceptions and doesn't force you to declare that it may throw an Exception
- Java has no equivalent of keyword arguments
- Java has no equivalent of **kwargs
- Java's equivalent of *args, is a variadic arg (eg `void static main(String... args)`)
    - but java doesn't have an equivalent of unpacking

I will go over other differences like lambdas and types in a later lesson

## Functions with default parameters

The next functions we will look at have a default parameter

In [ ]:
def die_w_default(size: int = 20) -> int:
    """Returns a random number from [1, size] inclusive

    Parameters
    ----------
    size : int, optional
        the die size, by default 20

    Returns
    -------
    int
        the result of the roll
    """
    return die(size)

def dice(number: int, size: int = 20) -> list[int]:
    """Rolls `number` of dice of type `size`

    Parameters
    ----------
    number : int
        the number of dice to roll
    size : int, optional
        the size of the die type, by default 20

    Returns
    -------
    list[int]
        the result of the die rolls
    """
    # this is called a list comprehension.  it is defined as:
    # [return_val for val in some_sequence]
    # it is equivalent to
    #
    # collection = []
    # for val in some_iterable:
    #     collection.append(val)
    return [die(size) for _ in range(number)]

### How defaulted params work

notice that after the `{name}: {type}` there is `= 20`.  This is how one defines a default value for a function or
method.  Using default args comes with some caveats:

- A defaulted arg must come after all non-defaulted args (see `dice` function)
- Do not use a default arg which is a mutable reference (eg, an empty list)

The latter caveat bears some explanation.  In python, the default argument is created **once** when the function is 
defined.  It is not generated each time a function is invoked.  Let's see what happens if we try

In [ ]:
# Can call it without naming the variable
print(f"Calling die_w_default(10) = {die_w_default(10)}")
# Or with naming it
print(f"Calling die_w_default(size=12) = {die_w_default(size=12)}")
die_w_default()

# same with dice
print(f"Rolling 3d6 = {dice(3, 6)}")
print(f"Rolling 4d20 = {dice(number=4, size=20)}")
# However, if you name a variable, it must come positional only args
#print(dice(number=4, 20)) # won't work
print(dice(4, size=20))

## Common mistake: default value is mutable

A common mistake in python is to create a default value which is a mutable reference

- what you think happens is on each invocation it is getting a new default value
- what actually happens is python assigns the default value once when it instantiates the function

In [ ]:
def mutable_default(num_rolls: int, rolls: list[int] = []) -> list[int]:
    for _ in range(num_rolls):
        result = dice(3, 6)
        rolls.extend(result)
    return rolls

# Let's make 1 roll
rolls = mutable_default(1)
print(rolls)
# and run again using the default
second_roll = mutable_default(1)
print(second_roll) # hmm, why does it contain the second?

third = mutable_default(1, [])
print(third)

In [ ]:
# Fixed version
def fixed_mutable_default(num_rolls: int, rolls: list[int] | None = None) -> list[int]:
    if rolls is None:
        rolls = []
    for _ in range(num_rolls):
        result = dice(3, 6)
        rolls.extend(result)
    return rolls

# Let's make 1 roll
rolls = fixed_mutable_default(1)
print(rolls)
# and run again using the default
second_roll = fixed_mutable_default(1)
print(second_roll) 

## Postional and keyword only arguments

Sometimes, when there are many arguments to a function or method, it's good to specify which arguments are positional
and which arguments must be specified by keyword.  As shown from the examples above, you could call any of the functions
either positionally, or with the name of the keyword.  To change this behavior you can do this:

- Use `/` in your argument list to specify that any _args to the left_ are positional only
- Use `*` in your argument list to specify that any _args to the right_ are keyword only

Let's modify dice so that you must specify the `number` as a positional arg, and size as a keyword arg

In [ ]:
def dice(number: int, /, *, size: int = 20) -> list[int]:
    return [die(size) for _ in range(number)]

#dice(number=4, size=10) # WRONG: can not specify number as a keyword arg
#dice(4, 10) # WRONG: must specify size as a keyword
dice(4, size=10)

## *args and **kwargs

Now that you know what positional and keyword args are, we can talk about *args, and **kwargs in python. A common
convention in python code is to specify all positional arguments as `*args` and all keyword arguments as `**kwargs`.
While this is a common convention, the only place this should be used in modern python is when creating decorator 
functions (which will be a topic for a more advanced session).

- `*args` is actually a list of arguments that are passed to the function
- `**kwargs` is actually a dict, where the key is the name of the argument, and the value is whatever the value is.

The problem with `*args` and especially `**kwargs` is knowing what the meaning of the arguments are.  Sadly, a lot of
legacy code used these conventions and paid the price for it

> I used to work at Red Hat on the Openstack nova team (the compute side) and 9 years ago, it was 3 million lines of 
code.  We saw so many TypeError or KeyError bugs it wasn't funny.  This is because too many devs were lazy and just
created methods like `def foo(requird_arg1, *args, required_named_arg=10, **kwargs):`.  When you write code like this,
not only do you not know what the types are, you don't even know how many arguments are required, and what the key 
values (the names) in the `kwargs` dict are.  So, you are forced to look at the source code to figure out what is 
actually supposed to be passed in

In [ ]:
def bad_function(required_pos: int, *args, required_name: str, **kwargs):
    print(f"I am a required positional argument with value: {required_pos}")
    print(f"These are all the rest of the positional args: {args}")
    print(f"I am a required_named arg: {required_name}")
    named_args = [f"{k} = {v}" for k, v in kwargs.items()]
    print(f"and these are all the other named arguments: {named_args}")

bad_function(10, "hi", "sean", required_name="toner", **{"one": 1, "two": "TWO"})

# example of unpacking *
args = (1, 2, 3)
print(f"\nCalling with star unpacking")
bad_function(10, *args, required_name="toner", **{"one": 1, "two": "TWO"})

obj = {"a": 1, "b": 2}
print(f"\nCalling with ** unpacking")
bad_function(10, "hi", "sean", required_name="toner", **obj)

print(f"\nCalling with extra keyword args")
bad_function(10, "hi", "sean", required_name="toner", brand_new_arg = 10, another_kw_arg = [1, 2, 3])

### When to use *args and **kwargs: Passing lists and dicts

Sometimes, the amount of arguments in a function becomes very large, and it is easier to pass in the arguments either
by a list (for positional args) or by a dict (for named arguments).  This is sometimes useful when you have a class
with many fields that need to be initialized as is often the case when using a `dataclass` type.

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Tuple


def lots_of_args(
    mapping: dict,
    answers: Tuple[int, int],
    cwd: str | None,
    completed: bool,
    start: datetime,
    end: datetime,
    timeout: timedelta
):
    print(mapping)
    print(answers)
    print(cwd)
    print(completed)
    print(start)
    print(end)
    print(timeout)
    return end + timeout

args = [
    {1: "foo"},
    (100, 200),
    None
]
kwargs = {
    "completed": True,
    "start": datetime.now(),
    "end": datetime.now(),
    "timeout": timedelta(hours=1)
}
lots_of_args(*args, **kwargs)


## Scopes

This will be a very brief intro, but hopefully even people using python will learn a thing or two here.  The first thing
we will go over is the concept of `scopes`.  Python, unlike most modern languages is not lexically scoped.

What does `lexically scoped` mean?  Lexical means essentially, words, but it has to do with "boundaries" of your code.
In many languages with curly braces, the braces define a scope.  Variables or other symbols introduced in that scope
(eg the curly braces) only last inside the brace section.

Python is not like that.  First off, it doesn't have curly braces.  But it does have indentation, so you might think
that the indentation acts as a kind of scope, just like curly braces do.  This is incorrect.  Python really only has 
Method, Class and Module scope (and with 3.12, a new generic scope).

The code below illustrates how 

In [ ]:
module_scoped = 10

def lookup():
    module_scoped = 100
    print(f"In function scope: module_scoped = {module_scoped}")

    # people used to lexical scoping think this is a bug.  They think `msg` lives in a scope defined by the if/else
    # block, and doesn't exist outside the if/else block.  Python only has a single scope at the function/method level
    if module_scoped < 100:
        msg = "value wasless than 100"
    else:
        msg = "value is greater than or equal to 100"
    print(msg)

lookup()
print(f"module_scoped is {module_scoped}") # it's still 10


In [ ]:
# Only a few things are immutable in python, like str, tuples, int, float.A list is mutable
a_list = [1, 2]

# Because a_list is a mutable reference and it's in the global (top-level) scope, the call here will mutate a_list
def captured_a_list(val: int):
    # Uncomment this to see what happens
    #a_list = [1, 2]
    a_list.append(val)
    print(a_list)

captured_a_list(10)
print(a_list)

In [12]:
def replacer(some_list: list[int]):
    print(f"id of some_list = {id(some_list)}")
    some_list = []
    print(f"id of some_list after assignment is now = {id(some_list)}")
    print(f"but id of a_list is still = {id(a_list)}")

print(f"id of a_list is {id(a_list)}")
replacer(a_list) # this is like lookup(), it creates a new binding, leaving the original alone
print(a_list)

id of a_list is 1828966114176
id of some_list = 1828966114176
id of some_list after assignment is now = 1828992152064
but id of a_list is still = 1828966114176
[1, 2]


## Namespaces

So far, I haven't talked about namespaces at all, but they play a critical role in python but they can easily trip even
intermediate or advanced users because usually scopes are implicit, and so it's not obvious what they are.

Scopes and namespaces go hand in hand with each other.  A namespace is a mapping of a symbol name to a value, for 
example, if a function uses a variable called `amount` in a calculation, python has to look up what `amount` is.  This
value `amount` may have been declared in one of many _scopes_.  

- module (global tothe module)
- function
- typing (introduced in 3.12)

If you look carefully at what happened in the replacer function, you may have been puzzled.  Why did `some_list` get
a new memory address?  Why didn't it just empty out what it was pointing to?

We can kind of think of a namespace like a map that python looks up:

```yaml
global:
  a_list:
    value: [1, 2]
    address: 1828966114176
function:
  replacer:
    some_list:
      value
```

## The global and nonlocal keywords

Though rarely used, sometimes it is necessary to mark a variable in a non top-level scope (ie, declared inside of a 
function, method, or class) as being either `global` or `nonlocal`.

Let's look at an example:

In [ ]:
def globalizer():
    global foo
    foo = "testing"

globalizer()
# Where did `foo` come from?  It got inserted into the global namespace through the `global` keyword
print(f"foo = {foo}")

def change_foo():
    foo = "hello"

change_foo()
print(f"foo did not change and is still = {foo}")

### A mutable global

As we can see, we tried to reassign the `global` variable foo inside of `change_foo` function, but it didn't work.  Is
there a way to make it work?

There is, but you have to create the variable

In [13]:
name = "sean"

def non_global_rename(to: str):
    name = to

non_global_rename("john")
print(f"without global name, name is still = {name}")

without global name, name is still = sean


In [14]:
def rename(to: str):
    global name
    name = to

print(f"Before call to rename, name = {name}")
rename("john") # because we use global, we reassign it
print(f"after call to rename, name is now = {name}")

Before call to rename, name = sean
after call to rename, name is now = john


In [17]:
def wrapper(name: str):
    def inner(greet: str):
        #nonlocal name # try uncommenting this
        name = "sean"
        return f"{greet} {name}"
    print(inner("hi"))
    return name

name = "tony"
print(wrapper(name))

hi sean
tony


## More scoping gotchas

This is a common source of confusion for new pythonistas

In [23]:
score = 75 # try setting this to 50 and uncommenting the other lines

def grader(check: int):
    # dont have to create a "default" grade here,  grade is scoped to the entire function
    # however, try changing score to 50, and commenting out the marked lines
    if check > 90:
        grade = "A"
    elif check > 80:
        grade = "B"
    elif check > 70:
        grade = "C"
    elif check > 60:
        grade = "D"
    #else:              # try commenting this out
    #    grade = "F"    # and this
    return grade


grader(score)

'C'

## Closures and Functions

Understanding scopes is necessary to make use of closures and nested functions.  We already showed several examples
of functions